In [3]:
# Required installations (run these commands in your terminal or command prompt)
# pip install convokit spacy nltk seaborn

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.sentiment import SentimentIntensityAnalyzer
import spacy
from convokit import Corpus, download
import string
from collections import defaultdict
import json


In [4]:
# Download necessary NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('vader_lexicon')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [5]:
# Load spaCy English language model
nlp = spacy.load('en_core_web_sm')

# Initialize NLTK tools
lemmatizer = WordNetLemmatizer()
sia = SentimentIntensityAnalyzer()

In [6]:
def preprocess_text(text):
    """
    Preprocesses the input text by converting to lowercase, removing punctuation, tokenizing,
    removing stopwords, and lemmatizing the words.
    """
    # Convert text to lowercase and remove punctuation
    text = text.lower().translate(str.maketrans('', '', string.punctuation))

    # Tokenize text
    tokens = word_tokenize(text)

    # Remove stopwords and lemmatize
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stopwords.words('english')]

    return ' '.join(tokens)

In [7]:
def analyze_text(text):
    """
    Analyzes the text for part-of-speech tags, sentiment, and named entities.
    """
    # POS Tagging
    pos_tags = nltk.pos_tag(word_tokenize(text))

    # Sentiment Analysis
    sentiment = sia.polarity_scores(text)

    # Named Entity Recognition with spaCy
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]

    return pos_tags, sentiment, entities

In [8]:
# Function to extract character names using NER
def extract_character_names(text):
    doc = nlp(text)
    return [ent.text for ent in doc.ents if ent.label_ == 'PERSON']

In [9]:
# Load corpus
corpus = Corpus(filename=download("friends-corpus"))

Dataset already exists at C:\Users\dell\.convokit\downloads\friends-corpus


In [10]:
# Structure to hold the dialogue count
seasons_structure = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))

In [11]:
# Dictionary to store interactions between characters
interactions = defaultdict(int)

In [12]:
sentiment_structure = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(int))))

In [13]:
# Inspect the first few conversations to understand the metadata structure
for conversation_id in corpus.get_conversation_ids()[:5]:  # Check the first 5 conversations
    conversation = corpus.get_conversation(conversation_id)
    print(conversation_id, conversation.meta)

s01_e01_c01_u001 ConvoKitMeta({'season': 's01', 'episode': 'e01', 'scene': 'c01'})
s01_e01_c02_u001 ConvoKitMeta({'season': 's01', 'episode': 'e01', 'scene': 'c02'})
s01_e01_c03_u001 ConvoKitMeta({'season': 's01', 'episode': 'e01', 'scene': 'c03'})
s01_e01_c04_u001 ConvoKitMeta({'season': 's01', 'episode': 'e01', 'scene': 'c04'})
s01_e01_c05_u001 ConvoKitMeta({'season': 's01', 'episode': 'e01', 'scene': 'c05'})


In [14]:
main_characters = [
    "Joey Tribbiani", 
    "Monica Geller", 
    "Ross Geller", 
    "Rachel Green", 
    "Phoebe Buffay", 
    "Chandler Bing"
]

In [15]:
# Initialize the character development structure with characters, seasons, and episodes
character_development = {character: {} for character in main_characters}

In [16]:
# Process and analyze text in the corpus
for conversation_id in corpus.get_conversation_ids():
    conversation = corpus.get_conversation(conversation_id)
    season_id = conversation.meta['season']
    episode_id = conversation.meta['episode']
    for utterance_id in conversation.get_utterance_ids():
        utterance = corpus.get_utterance(utterance_id)
        characters = extract_character_names(utterance.text)
        # Preprocess the text of each utterance
        processed_text = preprocess_text(utterance.text)
         # Analyze the preprocessed text for POS, sentiment, and NER
        pos_tags, sentiment, entities = analyze_text(processed_text)
        # Example of how to use the processed data
        # print(f"Utterance ID: {utterance.id}), POS Tags: {pos_tags}, Sentiment: {sentiment}, Entities: {entities}")
        # Extract characters from the utterance
        characters = extract_character_names(utterance.text)
        # Count interactions between all pairs of characters in the utterance
        #for i in range(len(characters)):
            #for j in range(i + 1, len(characters)):
                #pair = tuple(sorted([characters[i], characters[j]]))
                #interactions[pair] += 1

        #for character in characters:
            # Count words spoken by each character in each episode
            # seasons_structure[season_id][episode_id][character] += len(utterance.text.split())        

        sentiment = sia.polarity_scores(utterance.text)
        #for character in characters:
            #sentiment_structure[season_id][episode_id][character]['positive'] += sentiment['pos']
            #sentiment_structure[season_id][episode_id][character]['negative'] += sentiment['neg']
            #sentiment_structure[season_id][episode_id][character]['neutral'] += sentiment['neu']

        for character in main_characters:  # main_characters is a list of your main characters
            if season_id not in character_development[character]:
                character_development[character][season_id] = {}
            if episode_id not in character_development[character][season_id]:
                character_development[character][season_id][episode_id] = {
                'positive': 0,
                'negative': 0,
                'neutral': 0,
                'word_count': 0,  # You might want to track word count or other metrics
            }

        for character in characters:
            if character in main_characters:  # Only consider main characters
                char_data = character_development[character][season_id][episode_id]
                char_data['positive'] += sentiment['pos']
                char_data['negative'] += sentiment['neg']
                char_data['neutral'] += sentiment['neu']
                char_data['word_count'] += len(utterance.text.split())    

In [ ]:
# After the loop, prepare the data for D3.js
nodes = list({name for pair in interactions for name in pair})
links = [{"source": pair[0], "target": pair[1], "value": count} for pair, count in interactions.items()]

# Save the processed data to JSON files
with open('static/nodes.json', 'w') as f:
    json.dump([{"id": name} for name in nodes], f)

with open('static/links.json', 'w') as f:
    json.dump(links, f)

In [14]:
with open('static/dialogue_distribution.json', 'w') as f:
    json.dump(dict(seasons_structure), f)

In [ ]:
with open('static/sentiment_analysis.json', 'w') as f:
    json.dump(dict(sentiment_structure), f)

In [ ]:
# Save the character development data to a JSON file
with open('static/character_development.json', 'w') as f:
    json.dump(character_development, f, indent=4)

In [ ]:
# Print summary stats if needed
corpus.print_summary_stats()